# Exercícios

In [1]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tweets_classificados.csv", encoding='utf-8')
df.head()

,id,data_tweet,texto,sentimento
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,Neutro
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [2]:
df['sentimento'].unique()

array(['Neutro', 'Negativo', 'Positivo'], dtype=object)

## ToDo 1

Altere as funções de tratamento de texto apresentadas em sala para que elas façam a remoção de links também. 

Crie uma nova coluna chamada texto_tratado que conterá o resultado da aplicação das funções. 

In [8]:
# resposta
import nltk
import re
import string
import unicodedata

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
    
        
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def normalize_str(text):
    text = text.lower()
    text = re.sub("(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", " ",text) #remove links
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text


def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words("english") # portuguese, caso o dataset seja em português
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None


In [9]:
df['texto_tratado'] = df['texto'].apply(tokenizer)

In [10]:
df.head()

,id,data_tweet,texto,sentimento,texto_tratado
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro,catedral santo antonio governador valadares
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro,governador valadares minas gerais
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro,governador valadares minas gerais
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,Neutro,
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo,psol vai questionar aumento vereadores prefeit...


## ToDo 2

Ao fazer a remoção de links, percebemos que algumas linhas da coluna texto_tratado possuem valores faltantes. Entretanto, o Python trata eles como ''(str) e nao como Null. Assim, um simples dropna nao resolve o problema. 

Encontre uma forma de remover tais elementos. Dica: use o índice das linhas cujos elementos da coluna texto_tratado seja nulo. 

In [29]:
#resposta
df = df.drop(df.index[[list(np.where(df['texto_tratado']==''))[0]]]).reset_index()
df.head()

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:5055: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


,index,id,data_tweet,texto,sentimento,texto_tratado
0,0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro,catedral santo antonio governador valadares
1,1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro,governador valadares minas gerais
2,2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro,governador valadares minas gerais
3,4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo,psol vai questionar aumento vereadores prefeit...
4,5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",Neutro,bom bandido morto deputado cabo julio condenad...


## ToDo 3

Separe a coluna texto_tratado em conjunto de treino e teste na proporção 70/30

In [30]:
from sklearn.model_selection import train_test_split

# divide o dataframe em amostras de treino e teste
df_train, df_test = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

## ToDo 4

Transforme os dados para criar a representação numérica dos textos. Use uma versão com CountVectorizer e outra com TFIDFVectorizer

In [33]:
# resposta - CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,2))   
cv.fit(df_train.texto_tratado)
x_train_cv = cv.transform(df_train.texto_tratado)

In [34]:
# resposta - TFIDFVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,1))   
tfidf.fit(df_train.texto_tratado)
x_train_tf = tfidf.transform(df_train.texto_tratado)

## ToDo 5

Treine uma árvore de decisão nas duas abordagens e compare seus resultados

In [35]:
# resposta - CountVecorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

y_train = df_train["sentimento"]
tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train_cv, y_train)

x_test = cv.transform(df_test.texto_tratado)
y_prediction = tree.predict(x_test)

y_test = df_test["sentimento"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

0.9247685185185185


In [36]:
# resposta - TFIDFVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

y_train = df_train["sentimento"]
tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train_tf, y_train)

x_test = tfidf.transform(df_test.texto_tratado)
y_prediction = tree.predict(x_test)

y_test = df_test["sentimento"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

0.9085648148148148


## ToDo 6
Crie uma função que lematiza as palavras da coluna texto_tratado apenas se elas forem um verbo. Depois, crie uma nova coluna chamada texto_tratado_lemma que conterá o resultado da aplicação da função na coluna texto_tratado. 

Dica: use o Corpus pt_core_news_sm como referência para determinar a classe gramatical da palavra

In [39]:
# resposta
import spacy
nlp = spacy.load('pt_core_news_sm')
def lemmatizer_verbs(text):
    sent = []
    doc = nlp(text)
    for word in doc:
        if word.pos_ == "VERB":
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)

In [40]:
df['texto_tratado_lemma'] = df.texto_tratado.apply(lemmatizer_verbs)

In [41]:
df.head()

,index,id,data_tweet,texto,sentimento,texto_tratado,texto_tratado_lemma
0,0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro,catedral santo antonio governador valadares,catedral santo antonio governador valadares
1,1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro,governador valadares minas gerais,governador valadar minas gerais
2,2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro,governador valadares minas gerais,governador valadar minas gerais
3,4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo,psol vai questionar aumento vereadores prefeit...,psol vai questionar aumento vereadores prefeit...
4,5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",Neutro,bom bandido morto deputado cabo julio condenad...,bom bandido morrer deputado cabo julio condena...


## ToDo 7

repita os ToDo 3, ToDo 4 e ToDo 5, usando como feature a coluna texto_tratado_lemma, e veja se os resultados tiveram melhora.

In [43]:
df_train, df_test = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

In [47]:
#resposta - CountVectorizer
cv2 = CountVectorizer(ngram_range=(1,1))   
cv2.fit(df_train.texto_tratado_lemma)
x_train_cv2 = cv2.transform(df_train.texto_tratado_lemma)

In [48]:
# resposta - TFIDFVectorizer
tfidf2 = TfidfVectorizer(ngram_range=(1,1))  
tfidf2.fit(df_train.texto_tratado_lemma)
x_train_tf2 = tfidf2.transform(df_train.texto_tratado_lemma)

In [50]:
y_train = df_train["sentimento"]
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train_cv2, y_train)

x_test = cv2.transform(df_test.texto_tratado_lemma)

y_prediction = model.predict(x_test)

y_test = df_test["sentimento"]
accuracy = accuracy_score(y_prediction, y_test)
print('Acurácia com CV: ',accuracy)

Acurácia com CV:  0.9259259259259259


In [51]:
y_train = df_train["sentimento"]
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train_tf2, y_train)

x_test = tfidf2.transform(df_test.texto_tratado_lemma)

y_prediction = model.predict(x_test)

y_test = df_test["sentimento"]
accuracy = accuracy_score(y_prediction, y_test)
print('Acurácia com TFIDF: ',accuracy)

Acurácia com TFIDF:  0.9033564814814815
